# Financial Inclusion Analysis & Forecasting Walkthrough

This notebook demonstrates the end-to-end workflow for analyzing historical financial inclusion data in Ethiopia and generating event-adjusted forecasts.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add parent directory to path to import src modules
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.loader import load_data
from src.forecast import generate_baseline_forecast, apply_event_impacts

## 1. Load Data
We use the custom loader to ingest the unified Excel dataset.

In [ ]:
data = load_data('../data/raw/ethiopia_fi_unified_data.xlsx')
obs = data['observations']
events = data['events']
impacts = data['impacts']

print(f"Loaded {len(obs)} observations and {len(events)} events.")
obs.head()

## 2. Exploratory Data Analysis (EDA)
Visualizing the historical trends of key indicators.

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=obs, x='observation_date', y='value_numeric', hue='indicator', marker='o')
plt.title('Historical Financial Inclusion Indicators')
plt.grid(True)
plt.show()

## 3. Forecasting
We generate a baseline forecast using linear regression and then adjust it based on future policy events.

In [ ]:
indicators = obs['indicator'].unique()
forecast_years = [2025, 2026, 2027, 2028]
all_forecasts = []

# Generate Baselines
for ind in indicators:
    baseline = generate_baseline_forecast(obs, ind, forecast_years)
    if not baseline.empty:
        all_forecasts.append(baseline)

baseline_df = pd.concat(all_forecasts, ignore_index=True)

# Apply Impacts
if 'impact_estimate' in impacts.columns:
    impacts['impact_estimate'] = pd.to_numeric(impacts['impact_estimate'], errors='coerce').fillna(0)

final_forecast = apply_event_impacts(baseline_df, events, impacts, pd.Timestamp('2024-01-01'))

final_forecast.head()

## 4. Visualize Forecasts
Comparing the baseline trend against the event-adjusted scenario.

In [ ]:
target_indicator = 'Account Ownership Rate'
subset = final_forecast[final_forecast['indicator'] == target_indicator]

plt.figure(figsize=(10, 6))
sns.lineplot(data=subset, x='observation_date', y='value_numeric', hue='type', style='type', markers=True)
plt.title(f'Forecast Scenarios: {target_indicator}')
plt.grid(True)
plt.show()